## Synergos (Abalone dataset)

This notebook serves as a guide to perform an entire federated cycle in three different phases with two worker nodes.

**PHASE 1**. Connect \
**PHASE 2**. Train \
**PHASE 3**. Evaluate

Ensure the TTP, worker_1 and worker_2 containers are already running, otherwise run the following node in sequential.

1. **Run worker_1 node**:

`$ pysyft_worker > docker run -v path_to_abalone_dataset/abalone/data1:/worker/data -v 
path_to_outputs/outputs_1:/worker/outputs --name worker_1 worker:pysyft_demo`

2. **Run worker_2 node**:

`$ pysyft_worker > docker run -v path_to_abalone_dataset/abalone/data1:/worker/data -v path_to_outputs/outputs_2:/worker/outputs --name worker_2 worker:pysyft_demo`

3. **Run TTP node**:

`$ pysyft_ttp > docker run -p 5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v path_to_ttp_data/ttp_data:/ttp/data -v path_to_ml_flow_directory/mlflow_test:/ttp/mlflow --name ttp --link worker_1 --link worker_2 ttp:pysyft_demo`


## Initializing Synergos driver

In [1]:
from synergos import Driver

host = "0.0.0.0"
port = 5000

driver = Driver(host=host, port=port)

## Phase 1: CONNECT - Submitting TTP & Participant metadata #

#### 1A. TTP controller creates a project

Create a project which is a classification task with action="classify", if the project is a regression task thenn action="regress".

In [3]:
# 1A. TTP controller creates a project
driver.projects.create(
    project_id="test_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': []
    }
)

{'data': {'doc_id': '1',
  'kind': 'Project',
  'key': {'project_id': 'test_project'},
  'relations': {'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'action': 'classify',
  'incentives': {'tier_2': [], 'tier_1': []}},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'projects.post',
 'params': {}}

#### 1B. TTP controller creates an experiment

In [4]:
# 1B. TTP controller creates an experiment
driver.experiments.create(
    project_id="test_project",
    expt_id="test_experiment",
    model=[
        {
            "activation": "sigmoid",
            "is_input": True,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 28,
                "out_features": 1
            }
        }
    ]
)

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'project_id': 'test_project'},
 'data': {'created_at': '2020-09-23 03:25:04 N',
  'key': {'expt_id': 'test_experiment', 'project_id': 'test_project'},
  'model': [{'activation': 'sigmoid',
    'is_input': True,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 28, 'out_features': 1}}],
  'relations': {'Run': [], 'Model': [], 'Validation': [], 'Prediction': []},
  'doc_id': 1,
  'kind': 'Experiment'}}

#### 1C. TTP controller creates a run

In [6]:
driver.runs.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss"
)

{'data': {'doc_id': '1',
  'kind': 'Run',
  'key': {'project_id': 'test_project',
   'expt_id': 'test_experiment',
   'run_id': 'test_run'},
  'relations': {'Model': [], 'Validation': [], 'Prediction': []},
  'rounds': 2,
  'epochs': 1,
  'lr': 0.001,
  'lr_decay': 0.1,
  'weight_decay': 0.0,
  'seed': 42,
  'precision_fractional': 5,
  'mu': 0.1,
  'l1_lambda': 0.0,
  'l2_lambda': 0.0,
  'base_lr': 0.0005,
  'max_lr': 0.005,
  'patience': 10,
  'delta': 0.0},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'runs.post',
 'params': {'project_id': 'test_project', 'expt_id': 'test_experiment'}}

#### 1D. Participants registers their servers' configurations

In [7]:
driver.participants.create(
    participant_id="test_participant_1",
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

driver.participants.create(
    participant_id="test_participant_2",
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'test_participant_2'},
  'relations': {'Registration': [], 'Tag': []},
  'id': 'test_participant_2',
  'host': '172.17.0.3',
  'port': 8020,
  'log_msgs': True,
  'verbose': True,
  'f_port': 5000},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'participants.post',
 'params': {}}

#### 1E. Participants registers their role in a specific project

In [8]:
driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_1",
    role="guest"
)

driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_2",
    role="host"
)

{'data': {'doc_id': '2',
  'kind': 'Registration',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'relations': {'Tag': []},
  'project': {'incentives': {'tier_2': [], 'tier_1': []}, 'start_at': None},
  'participant': {'id': 'test_participant_2',
   'host': '172.17.0.3',
   'port': 8020,
   'f_port': 5000,
   'log_msgs': True,
   'verbose': True},
  'role': 'host'},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {}}

#### 1F. Participants registers their tags for a specific project

In [9]:
driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_1",
    train=[
        ["train"]
    ],
    evaluate = [["evaluate"]]
)

driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_2",
    train=[["train"]]
)

{'data': {'doc_id': '2',
  'kind': 'Tag',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'train': [['train']],
  'evaluate': [],
  'predict': []},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'tags.post',
 'params': {'project_id': 'test_project',
  'participant_id': 'test_participant_2'}}

## Phase 2: TRAIN - Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [10]:
driver.alignments.create(project_id="test_project")

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'project_id': 'test_project'},
 'data': [{'created_at': '2020-09-23 03:25:14 N',
   'key': {'participant_id': 'test_participant_2',
    'project_id': 'test_project'},
   'link': {'alignment_id': '654af060fd4c11ea8a1d0242ac110004',
    'registration_id': '63b78ceafd4c11ea9b610242ac110004',
    'tag_id': '64256af8fd4c11ea93f10242ac110004'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 1,
   'kind': 'Alignment'},
  {'created_at': '2020-09-23 03:25:14 N',
   'evaluate': {'X': [],
    'y': [0,
     1,
     2,
     3,
     4,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27]},
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project'},
   'link': {'alignment_id': '654e40a8fd4c11ea8a1d0242ac110004',
    'registration_id': '63adc962fd4c11eab0180242ac110004',
  

#### 2B. Trigger training across the federated grid

In [11]:
model_resp = driver.models.create(
        project_id="test_project",
        expt_id="test_experiment",
        run_id="test_run")

## Phase 3: EVALUATE - Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [12]:
driver.validations.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Validation',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'evaluate': {'statistics': {'accuracy': [0.42857142857142855,
      0.5357142857142857,
      0.6071428571428571],
     'roc_auc_score': [0.7083333333333334,
      0.5130434782608696,
      0.47619047619047616],
     'pr_auc_score': [0.41577380952380955,
      0.1713714716655893,
      0.21569130583322071],
     'f_score': [0.11111111111111112, 0.13333333333333333, 0.0],
     'TPRs': [0.25, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0],
     'TNRs': [0.4583333333333333,
      1.0,
      1.0,
      1.0,
      0.6521739130434783,
      1.0,
      1.0,
      0.8571428571428571],
     'PPVs': [0.07142857142857142, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0],
     'NPVs': [0.7857142857142857,
      0.8214285714285714,
      0.75,
      0.8928571428571429,
      0.8333333333333334,
      0.8928571428571429,
      0

#### 3B. Perform prediction(s) of combination(s)

In [13]:
driver.predictions.create(
    tags={"test_project": [["predict"]]},
    participant_id="test_participant_1",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Prediction',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'predict': {'statistics': {'accuracy': [0.42857142857142855,
      0.5357142857142857,
      0.6071428571428571],
     'roc_auc_score': [0.7083333333333334,
      0.5130434782608696,
      0.47619047619047616],
     'pr_auc_score': [0.41577380952380955,
      0.1713714716655893,
      0.21569130583322071],
     'f_score': [0.11111111111111112, 0.13333333333333333, 0.0],
     'TPRs': [0.25, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0],
     'TNRs': [0.4583333333333333,
      1.0,
      1.0,
      1.0,
      0.6521739130434783,
      1.0,
      1.0,
      0.8571428571428571],
     'PPVs': [0.07142857142857142, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0],
     'NPVs': [0.7857142857142857,
      0.8214285714285714,
      0.75,
      0.8928571428571429,
      0.8333333333333334,
      0.8928571428571429,
      0.